In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/Datasets/eeg-motor-movementimagery-dataset-1.0.0.zip" ./
!unzip "eeg-motor-movementimagery-dataset-1.0.0.zip"

Archive:  eeg-motor-movementimagery-dataset-1.0.0.zip
   creating: files/
   creating: files/S001/
   creating: files/S002/
   creating: files/S003/
   creating: files/S004/
   creating: files/S005/
   creating: files/S006/
   creating: files/S007/
   creating: files/S008/
   creating: files/S009/
   creating: files/S010/
   creating: files/S011/
   creating: files/S012/
   creating: files/S013/
   creating: files/S014/
   creating: files/S015/
   creating: files/S016/
   creating: files/S017/
   creating: files/S018/
   creating: files/S019/
   creating: files/S020/
   creating: files/S021/
   creating: files/S022/
   creating: files/S023/
   creating: files/S024/
   creating: files/S025/
   creating: files/S026/
   creating: files/S027/
   creating: files/S028/
   creating: files/S029/
   creating: files/S030/
   creating: files/S031/
   creating: files/S032/
   creating: files/S033/
   creating: files/S034/
   creating: files/S035/
   creating: files/S036/
   creating: files/S037/
 

<h2><b>Installing Required Packages</b></h2>

In [ ]:
!pip install mne
!pip install pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.3 MB/s eta 0:00:00


<h2><b>Importing Required Packages</b></h2>

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy
from numba import njit, prange
from pyts.image import RecurrencePlot

import pandas as pd

import os
import shutil

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.applications import ResNet50,MobileNetV2,NASNetMobile
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

<h2><b>Imporing and Pre-Processing the raw EEG signal Data and producing Recurrence Plot</b></h2>

In [ ]:
BASE_DIR = "files"
REQ_PERSONS = ["S001"
,"S002","S003","S004","S005","S006","S007","S008","S009","S010",
               "S011","S012","S013","S014","S015","S016","S017","S018","S019","S020",
               "S021","S022","S023","S024","S025","S026","S027","S028","S029","S030",
               "S031","S032","S033","S034","S035","S036","S037","S038","S039","S040",
               "S041","S042","S043","S044","S045","S046","S047","S048","S049","S050",
               "S051","S052","S053","S054","S055","S056","S057","S058","S059","S060",
               "S061","S062","S063","S064","S065","S066","S067","S068","S069","S070",
               "S071","S072","S073","S074","S075","S076","S077","S078","S079","S080",
               "S081","S082","S083","S084","S085","S086","S087","S088","S089","S090",
               "S091","S092","S093","S094","S095","S096","S097","S098","S099","S100",
               "S101","S102","S103","S104","S105","S106","S107","S108","S109"
               ]
REQ_EDFS = ["03","07","11"
            ,"05","06","09"
            # ,"04","08","12"
            ]


REQUIRED = ['Fc5.',
            'Fc1.', 'Fcz.', 'Fc2.', 'C3..', 'Cz..', 'C4..'
            ]
ANNOTS = ["T0","T1","T2"]

CLASSES = ["rest","left","right"]

In [ ]:
def find_channel_index(channel_names, required_channels):
    chn_index = []
    for i in range(len(channel_names)):
        if channel_names[i] in required_channels:
            chn_index.append(i)
    return chn_index

In [ ]:
def extract_annotation_wise_data(raw_data,data, desc, onset, duration):
    extracted_data = {'T0': [], 'T1': [], 'T2': []}
    for i in range(len(desc)):
        start = data.time_as_index([onset[i]])[0]
        end = data.time_as_index([onset[i]+4])[0]
        sample_array = raw_data[:, int((onset[i]) * 160)+20:int((onset[i] + int(duration[i])) * 160)-20]
        extracted_data[desc[i]].append(sample_array)
    return extracted_data

In [ ]:
def find_RP(raw_data):
    transformer = RecurrencePlot()
    rp_values = transformer.transform(raw_data)
    return rp_values

In [ ]:
def save_RP(rp_data, path):

    for i in range(len(REQUIRED)):
        values = rp_data[i]

        fig = plt.figure(frameon=False)
        fig.set_size_inches(w=(5,5))

        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)

        ax.imshow(values, cmap='hot', interpolation='nearest')

        save_path = path + str(i) + ".png"
        # print(save_path)
        fig.savefig(os.path.join("output",save_path),bbox_inches="tight")
        ax.clear()
        fig.clear()
        plt.cla()
        plt.clf()
        plt.close("all")


In [ ]:
def produce_save_RP(annot_data, path):

  for annot in ANNOTS:
    annot_path = path + annot + "_"
    num_event = 0

    for event_data in annot_data[annot]:
      # print(event_data)
      rp = find_RP(event_data)

      event_path = annot_path + str(num_event) + "_"
      save_RP(rp, event_path)
      # print(event_path)

      num_event += 1

In [ ]:
!mkdir output

In [ ]:
req_chns = []
for person_dir in REQ_PERSONS:

  for edf_file in os.listdir(os.path.join(BASE_DIR, person_dir)):

    if edf_file.endswith("edf") and edf_file[5:7] in REQ_EDFS:
      annot_wise_data = {}

      data = mne.io.read_raw_edf(os.path.join(BASE_DIR, person_dir, edf_file))

      if len(req_chns) == 0:
        channels = data.ch_names
        req_chns = find_channel_index(channels,REQUIRED)

      raw_data= data.get_data(picks=req_chns)
      # print(raw_data)

      annot_wise_data = extract_annotation_wise_data(raw_data,data, data.annotations.description, data.annotations.onset, data.annotations.duration)

      person_path = edf_file[:7] + "_"

      produce_save_RP(annot_wise_data, person_path)

  print(f"Finished {person_dir}")






Extracting EDF parameters from /content/files/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/files/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/files/S001/S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/files/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Finished S001
Extracting EDF parameters from /content/files/S002/S002R07.edf...
EDF file detected
Setting 

<h2><b>Run No.1</b></h2>

In [ ]:
os.mkdir("data")
os.mkdir("data/rest")
os.mkdir("data/left")
os.mkdir("data/right")
os.mkdir("data/both_h")
os.mkdir("data/both_l")

In [ ]:
TRAIN_DIR = "output"
for image in os.listdir(TRAIN_DIR):
  if image.split("_")[1] == "T0":
    shutil.copy(os.path.join(TRAIN_DIR, image),"data/rest")
  elif image.split("_")[1] == "T1" and image[5:7] in ["03","07","11"]:
    shutil.copy(os.path.join(TRAIN_DIR, image),"data/left")
  elif image.split("_")[1] == "T2" and image[5:7] in ["03","07","11"]:
    shutil.copy(os.path.join(TRAIN_DIR, image),"data/right")
  elif image.split("_")[1] == "T1" and image[5:7] in ["05","06","09"]:
    shutil.copy(os.path.join(TRAIN_DIR, image),"data/both_h")
  elif image.split("_")[1] == "T2" and image[5:7] in ["05","06","09"]:
    shutil.copy(os.path.join(TRAIN_DIR, image),"data/both_l")

In [ ]:
IMG_HT = 128
IMG_WD = 128
CHANNELS = 3

DATA_DIR = "data/"

BATCH_SIZE = 32
EPOCHS = 8


In [ ]:
datagen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)

train = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HT, IMG_WD),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
    )

validation = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HT, IMG_WD),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
    )


Found 7336 images belonging to 5 classes.
Found 1834 images belonging to 5 classes.


In [ ]:
rnet_feature_extractor = NASNetMobile(include_top=False, weights="imagenet", input_shape=(IMG_HT,IMG_WD,CHANNELS))

NASNetMobile

out = rnet_feature_extractor.layers[-1].output
out = Flatten()(out)

for layer in rnet_feature_extractor.layers:
    layer.trainable = True

out = Dropout(0.05)(out)
out = Dense(512, activation="relu")(out)
out = Dropout(0.2)(out)
out = Dense(256, activation="sigmoid")(out)
out = Dropout(0.2)(out)
out = Dense(5,activation="softmax")(out)


rnet_transfer_model = Model(rnet_feature_extractor.input, out)


rnet_transfer_model.summary()

19993432/19993432 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 stem_conv1 (Conv2D)         (None, 63, 63, 32)           864       ['input_2[0][0]']             
                                                                                                  
 stem_bn1 (BatchNormalizati  (None, 63, 63, 32)           128       ['stem_conv1[0][0]']          
 on)                                                                                              
                                                                                                  
 activation (Activation)   

In [ ]:
adam = Adam(lr=0.01)

rnet_transfer_model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])


# rnet_transfer_model.fit_generator(
#     train,
#     validation_data = validation,
#     verbose=2,
#     epochs = EPOCHS)

# Alternatively, one could checkpoint just the model weights as -
checkpoint_filepath = '/content/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
rnet_transfer_model.fit_generator(
    train,
    validation_data = validation,
    verbose=2,
    epochs = EPOCHS,callbacks=[model_checkpoint_callback])


<ipython-input-15-7cf469b1757d>:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  rnet_transfer_model.fit_generator(


Epoch 1/8
230/230 - 1256s - loss: 1.4226 - accuracy: 0.4753 - val_loss: 1.4391 - val_accuracy: 0.5038 - 1256s/epoch - 5s/step
Epoch 2/8
230/230 - 1141s - loss: 1.3917 - accuracy: 0.4775 - val_loss: 1.5952 - val_accuracy: 0.1221 - 1141s/epoch - 5s/step
Epoch 3/8
230/230 - 1148s - loss: 1.3736 - accuracy: 0.4843 - val_loss: 1.5048 - val_accuracy: 0.5038 - 1148s/epoch - 5s/step
Epoch 4/8
230/230 - 1147s - loss: 1.3702 - accuracy: 0.4929 - val_loss: 1.4875 - val_accuracy: 0.5038 - 1147s/epoch - 5s/step
Epoch 5/8
230/230 - 1147s - loss: 1.3537 - accuracy: 0.4975 - val_loss: 1.5474 - val_accuracy: 0.5038 - 1147s/epoch - 5s/step
Epoch 6/8
230/230 - 1133s - loss: 1.3419 - accuracy: 0.4915 - val_loss: 1.4194 - val_accuracy: 0.5038 - 1133s/epoch - 5s/step
Epoch 7/8
230/230 - 1132s - loss: 1.3371 - accuracy: 0.4955 - val_loss: 1.6891 - val_accuracy: 0.5038 - 1132s/epoch - 5s/step
Epoch 8/8
230/230 - 1131s - loss: 1.3603 - accuracy: 0.5005 - val_loss: 1.4144 - val_accuracy: 0.5038 - 1131s/epoch - 